In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from types import SimpleNamespace
from scipy import sparse
from glob import glob
import re

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
sys.path.append(".")
from spham import *

from tictoc import TicToc

meV = units["meV"]
Tesla = units["T"]
mV = units["mV"]
μS = units["μS"]
nS = units["nS"]
Kelvin = units["K"]
nA = units["nA"]
pA = units["pA"]

In [ ]:
datadir = "data"

In [ ]:
class Experiment:
    def __init__(
        self,
        atoms, # atoms list
        Ji = {}, # coupling
        Bx = 0.0, # Field in 'x'
        By = 0.0, # Field in 'x'
        Bz = 0.0, # Field in 'x'
        maxstates = 500, # Number of states to calculate
        
    ):
        pass


In [ ]:
class Eigens:
    """Eigenvalues and eigenvectors of interacting atoms on a surface.
    
    Eigenvalues and eigenvectors of interacting atoms on a surface with possibility 
    to save on disk and reload.
    
    Attributes
    ----------
    En: numpy.array
       The eigenvalues in increasing order
       
    states: np.array
       The eigenvector ordered to match the eigenvalues
    """
    
    def __init__(self, 
                 wd, 
                 name,
                 atoms, 
                 Ji,
                 Bx,
                 By,
                 Bz,
                 maxstates = 1000,
                 createnewfile = False, # if previous calculation for more states create a file with less
                 inmemory = False,
                ):
        """Eigenvalues and eigenvectors of interacting atoms on a surface.

            Parameters:
            -----------
            wd: str
                Working directory where to read/write files
            name: str
                Name use in filename. When looking for previous calculation
                the name must match.
            atoms: iterable
                Iterable of SpinAtom or SpinOrbitAtom objects
            Ji: dict
                The coupling between atoms with tuple of index as key and coupling strengt as value.
                ex.: {(0, 1): 1.2*meV, (1,2): 1.3*meV}
            Bx: float
                Magnetic field in the 'x' direction
            By: float
                Magnetic field in the 'y' direction
            Bz: float
                Magnetic field in the 'z' direction
            maxstates: int
                Maximum number of states to calculate. Default: 1000
            createnewfile: bool
                If a previous calculation with more state is found, save a new file only
                including the lower number of states. Default: False
            inmemeory: bool
                Do not save calculation to disk. Useful for small system. Default: False

        """
        self.name = name
        self.wd = os.path.abspath(os.path.join(wd, self.name))
        self._inmemory = inmemory
        self.dim = len(atoms)
        
        self.params = {
            "atoms": atoms,
            "Ji": Ji,
            "Bx": Bx,
            "By": Bz,
            "Bz": Bz,
            "maxstates": maxstates,
        }
        
        if not inmemory:
            os.makedirs(self.wd, exist_ok=True)
            
        self.id = "{name:s}_{Bx:g}_{By:g}_{Bz:g}_{maxstates:d}".format(
            name = name,
            Bx = Bx/Tesla,
            By = By/Tesla,
            Bz = Bz/Tesla,
            maxstates = maxstates
        )
        
        self.fn = os.path.join(self.wd, "%s_eigens.npz" % self.id)
        
        docalc = True
        if self._inmemory:
            pass
        elif os.path.isfile(self.fn):
            if self.file_params == self.params:
                print("Previous calculation found for eigens in %s" % self.fn)
                docalc = False
        else:
            files = glob(os.path.join(self.wd, self.id + "*_eigens.npz"))
            nb = [int(f.split("_")[-2]) for f in files]
            
            for file, N in sorted(zip(files, nb), key=lambda x: x[1]):
                if N > maxstates:
                    print("%s calculated enough states already. Reusing it." % file)
                    docalc = False
                    if createnewfile:
                        t = TicToc()
                        t.tic("Creating new file from previous file.")
                        newfn = self.fn
                        self.fn = os.path.join(self.wd, file)
                        En=self.En
                        states=self.states
                        t.toc("Creating new file from previous file.")
                        self.fn = newfn
                        print("Saving file '%s'" % self.fn)
                        np.savez_compressed(self.fn, 
                                            En=En, 
                                            states=states, 
                                            params=[self.params,], 
                                            timing=[t.timing,],
                                           )
                    else:
                        self.fn = os.path.join(self.wd, file)
                    break
        if docalc: 
            self.calc()
            
    def calc(self):
        """ Energy and states calculation (eigenvalues and eigenvectors)"""
        t = TicToc()
        atoms = self.params["atoms"]
        Ji = self.params["Ji"]
        Bx = self.params["Bx"]
        By = self.params["By"]
        Bz = self.params["Bz"]
        maxstates = self.params["maxstates"]
        
        print("Calculation running in '%s'" % self.wd)
        t.tic("eigens_calc")
        
        t.tic("k (KronAtomSpace)")
        k = KronAtomSpace(atoms, precalc="loop") 
        t.toc("k (KronAtomSpace)")
        
        t.tic("hamiltonian")
        ham = hamcalc(k, Bx, By, Bz, Ji)
        t.toc("hamiltonian")
        
        t.tic("eigens")
        En, states = eigencalc(ham, maxstates)
        t.toc("eigens")
        
        t.toc("eigens_calc")
        if self._inmemory:
            self._En = En
            self._states = states
            self._timing = t.timing
            
        else:
            print("Saving file '%s'" % self.fn)
            np.savez_compressed(self.fn, 
                                En=En, 
                                states=states, 
                                params=[self.params,], 
                                timing=[t.timing,],
                               )
        
    @property
    def En(self):
        if self._inmemory:
            return self._En
        with np.load(self.fn, allow_pickle=True) as f:
            En = f["En"] 
            if En.shape[0] > self.params["maxstates"]:
                return En[:self.params["maxstates"]]
            else:
                return En
        
    @property
    def states(self):
        if self._inmemory:
            return self._states
        with np.load(self.fn, allow_pickle=True) as f:
            states = f["states"] 
            if states.shape[1] > self.params["maxstates"]:
                return states[:, :self.params["maxstates"]]
            else:
                return states
        
    @property
    def timing(self):
        if self._inmemory:
            return self._timing
        with np.load(self.fn, allow_pickle=True) as f:
            return f["timing"][0]
        
    def show_timing(self):
        print(TicToc(self.timing).nice_timing())
        
    @property
    def file_params(self):
        with np.load(self.fn, allow_pickle=True) as f:
            return f["params"][0]

In [ ]:
class YsYt:
    def __init__(self, 
                 eigens,
                 tippos,
                 u=0.0,
                 with_progress=False,
                 use_sparse=True,
                 inmemory=None,
                ):
        self.name = eigens.name
        self.wd = eigens.wd
        self._inmemory = inmemory if inmemory is not None else eigens._inmemory
        self.id = "_".join([
            eigens.id,
            str(tippos),
        ])
        self.fn = os.path.join(self.wd, "%s_Y.npz" % self.id)
        
        self.eigens = eigens
        self.params = eigens.params.copy()
        self.params["tippos"] = tippos
        self.params["u"] = u
        
        docalc = True
        if self._inmemory:
            pass
        elif os.path.isfile(self.fn):
            if self.file_params == self.params:
                print("Previous calculation found for Y (tippos=%d)" % tippos)
                docalc = False
        if docalc: self.calc(use_sparse=use_sparse, with_progress=with_progress)
        
    def calc(self, use_sparse=True, with_progress=False):
        t = TicToc()
        atoms = self.params["atoms"]
        maxstates = self.params["maxstates"]
        tippos = self.params["tippos"]
        u = self.params["u"]
        
        print("Calculation running in '%s'" % self.wd)
        t.tic("Y_calc")
        
        t.tic("ke (KronAtomSpace)")
        ke = KronAtomSpace(atoms, with_electron=True, precalc="loop") 
        t.toc("ke (KronAtomSpace)")
        
        t.tic("states (loading)")
        states = self.eigens.states
        t.toc("states (loading)")
        
        t.tic("Y")
        Y = Ycalc(tippos, 
                  ke, 
                  states, 
                  u, 
                  use_sparse=use_sparse, 
                  with_progress=with_progress
                 )
        t.toc("Y")
        
        
        t.toc("Y_calc")
        if self._inmemory:
            self._Ys = Y.s
            self._Yt = Y.t
            self._timing = t.timing
        else:
            print("Saving file '%s'" % self.fn)
            np.savez_compressed(self.fn, 
                                Ys=Y.s,
                                Yt=Y.t,
                                params=[self.params,], 
                                timing=[t.timing,],
                               )
        
    @property
    def Ys(self):
        if self._inmemory:
            return self._Ys
        with np.load(self.fn, allow_pickle=True) as f:
            return f["Ys"]
        
    @property
    def Yt(self):
        if self._inmemory:
            return self._Yt
        with np.load(self.fn, allow_pickle=True) as f:
            return f["Yt"]
        
    @property
    def timing(self):
        if self._inmemory:
            return self._timing
        with np.load(self.fn, allow_pickle=True) as f:
            return f["timing"][0]
        
    def show_timing(self):
        print(TicToc(self.timing).nice_timing())
        
    @property
    def file_params(self):
        with np.load(self.fn, allow_pickle=True) as f:
            return f["params"][0]

In [ ]:
class SteadyStates:
    def __init__(self, 
                 ysyt,
                 u,
                 η,
                 G0,
                 b0,
                 Gs,
                 bias,
                 temp,
                 algo,
                 inmemory=None,
                ):
        self.name = ysyt.name
        self.eigens = ysyt.eigens
        self.YsYt = ysyt
        self._inmemory = inmemory if inmemory is not None else ysyt._inmemory
        self.wd = os.path.abspath(ysyt.wd)
        self.id = "_".join([
            ysyt.id,
            "%g" % u,
            "%g" % η,
            "%g" % (G0 / μS),
            "%g" % b0,
            "%g" % (Gs / μS),
            "%g" % (temp / Kelvin),
            "%s" % algo,
        ])
        self.fn = os.path.join(self.wd, "%s_ss.npz" % self.id)
        
        self.params = ysyt.params.copy()
        self.params["u"] = u
        self.params["η"] = η
        self.params["G0"] = G0
        self.params["b0"] = b0
        self.params["Gs"] = Gs
        self.params["β"] = 1 / (units["kB"] * temp)
        self.params["algo"] = algo
        
        
        self.bias = bias
        
        docalc = True
        if self._inmemory:
            pass
        elif os.path.isfile(self.fn):
            with np.load(self.fn, allow_pickle=True) as f:
                if f["params"] == self.params:
                    tippos = self.params["tippos"]
                    print("Previous calculation found for steadystates at %d" % tippos)
                    docalc = False
        if docalc: self.calc()
        
    def calc(self):
        t = TicToc()
        
        t.tic("Steady states")
        
        donebias = self.donebias
        
        biases = [b for b in self.bias if b not in donebias]
        
        if len(biases) == 0:
            print("All biases already calculated")
            t.toc("Steady states")
            return
        
        algo = self.params["algo"]
        
        t.tic("R")
        R = self.R(*biases)
        t.toc("R")
        
        t.tic("ss")
        sss = []
        for r, bias in zip(R, biases):
            rij = r.ss + r.st + r.ts
            if algo.lower() == "rk23":
                sss.append(steadystatescalc_rk23(rij))
            elif algo.lower() == "null_space":
                sss.append(steadystatescalc_null_space(rij))
            elif algo.lower() == "nnls":
                sss.append(steadystatescalc_nnls(rij))
            elif algo.lower() == "root":
                sss.append(steadystatescalc_root(rij))
            elif algo.lower() == "firstorder":
                sss.append(steadystatescalc_firstorder(rij))
            elif algo.lower() == "randomwalk":
                sss.append(steadystatescalc_randomwalk(rij))
            else:
                print("%s not implemented" % algo)
        t.toc("ss")
        t.toc("Steady states")
        
        if os.path.isfile(self.fn):
            with np.load(self.fn, allow_pickle=True) as f:
                data = dict(**f)
        else:
            data = {"bias": [{}, ]}
            
        for bias, r, ss in zip(biases, R, sss):
            bstr = "%f" % bias
            data["bias"][0][bias] = bstr
            data["ss_" + bstr] = ss
            #data["Rst_" + bstr] = r.st
            #data["Rts_" + bstr] = r.ts
            data["params"] = [self.params,]
            data["timing"] = [t, ]
        
        if self._inmemory:
            self._data = data
        else:
            np.savez_compressed(self.fn, **data)
        
    @property
    def donebias(self):
        if not os.path.isfile(self.fn):
            return {}
        
        with np.load(self.fn, allow_pickle=True) as f:
            if f["params"] == self.params:
                return f['bias'][0]
        
    @property
    def file_params(self):
        with np.load(self.fn, allow_pickle=True) as f:
            return f["params"][0]
        
    @property
    def timing(self):
        with np.load(self.fn, allow_pickle=True) as f:
            return f["timing"][0]
        
    def show_timing(self):
        print(TicToc(self.timing).nice_timing())
        
    def bias_dict(self):
        if self._inmemory:
            return self._data["bias"][0]
        with np.load(self.fn, allow_pickle=True) as f:
            return f["bias"][0]
        
    def bias(self):
        return sorted(list(self.bias_dict.keys()))
    
    def R(self, *biases):
        En = self.eigens.En
        u = self.params['u']
        η = self.params['η']
        ys = self.YsYt.Ys
        yt = self.YsYt.Yt
        
        G0 = self.params["G0"]
        b0 = self.params["b0"]
        Gs = self.params["Gs"]
        β = self.params["β"]
        
        P = self.P()
        return [ratescalc(G0, b0, Gs, En, bias, β, P.ss, P.ts, P.st)
                for bias in biases]
    
    def P(self):
        u = self.params['u']
        η = self.params['η']
        ys = self.YsYt.Ys
        yt = self.YsYt.Yt
        P = Pcalc(ys, yt, η)
        return P
        
    def steadystates(self, *biases):
        return self._get("ss", biases)
        
    def _get(self, name, biases):
        d = self.bias_dict()
        if self._inmemory:
            f = self._data
            if len(biases) == 0:
                return f[name + "_" + d[biases[0]]]
            return [f[name + "_" + d[bias]] for bias in biases]
        with np.load(self.fn, allow_pickle=True) as f:
            if len(biases) == 0:
                return f[name + "_" + d[biases[0]]]
            return [f[name + "_" + d[bias]] for bias in biases]
    
    def statesdf(self):
        bias = self.bias
        return pd.DataFrame(self.steadystates(*bias), index=np.array(bias)/mV, copy=True)
    
    def dIdVdf(self):
        biases = self.bias
        steadystates = self.steadystates(*biases)
        R = self.R(*biases)
        b0 = self.params["b0"]
        G0 = self.params["G0"]
        c = []
        
        for bias, ss, r in zip(biases, steadystates, R):
            N = len(ss)
            c.append(np.sum(r.ts.T @ ss - r.st.T @ ss))
                
        c = np.array(c) + b0*G0*biases
        
        didv = np.gradient(c, biases)
        return pd.DataFrame(
            {"Current (nA)": c / nA, 
             "dIdV (μS)": didv/μS,
             "dIdV (nS)": didv/nS,
            }, index=pd.Index(biases/mV, name="Bias (mV)")
        )

In [ ]:
def showmat(mat, prec=0.001, wsum=False):
    m = np.atleast_2d(mat)
    
    mx = m.max()
    
    n1, n2 = m.shape
    
    cpx = np.any(np.iscomplex(m))
    
    print("┌" + "          "*n2 + "┐")
    for i in range(n1):
        print("│", end="")
        for j in range(n2):
            v = m[i, j] 
            if np.abs(np.real(v)) < prec:
                if np.abs(np.imag(v)) < prec:
                    print("     -    ", end='')
                else:
                    print(" %7.3fi " % np.imag(v), end='')
            else:
                print(" %7.3f  " % np.real(v), end='')
        print("│", end='')
        if cpx:
            print("\n│", end="")
            for j in range(n2):
                v = m[i, j] 
                if (np.abs(np.imag(v)) < prec) or (np.abs(np.real(v)) < prec):
                    print("          ", end='')
                else:
                    print("  %+7.3fi" % np.imag(v), end='')
            print("│", end='')
        if wsum == "conj":
            print(" ", end='')
            print(" %7.3f " % np.sum(np.conj(m[i, :]) @ m[:, i]), end='')
        elif wsum == "sum":
            print(" ", end='')
            print(" %7.3f " % np.sum(m[i, :]), end='')
        print()
    print("└" + "          "*n2 + "┘")
    
    if wsum == "conj":
        print(" ", end='')
        for i in range(n2):
            print(" %7.3f " % np.sum(np.conj(m[:, i]) @ m[:, i]), end='')
        print()
    elif wsum == "sum":
        print(" ", end='')
        for i in range(n2):
            print(" %7.3f " % np.sum(m[:, i]), end='')
        print("           %7.3f " % np.sum(m), end='')
        print()
            